In [8]:
import sys
sys.path.append('D:\PyCharmProjects\VFPUMC02')
sys.path.append(r'C:\Users\Administrator\PycharmProjects\VFPUMC02')
sys.path.append(r'/root/VFPUMC02')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from datasets.DataSet import BankDataset
from utils.DataProcessUtils import split_labeled_unlabeled,split_labeled_unlabeled_with_2_labels
from utils.DataProcessUtils import find_rounds_math
from utils.FateUtils import evaluate_model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from classfiers.TwoStep import TwoStep

In [10]:
bankDataset = BankDataset()

2025-01-21 19:33:31,504 - INFO - 正在加载数据集 bank.csv
2025-01-21 19:33:31,734 - INFO - 数据集 bank.csv 加载完成，数据集形状为 (41188, 64)


In [11]:
X, y = bankDataset.get_data()

In [12]:
X, y_C, y_R = X[:,:-1], y, X[:,-1]

In [13]:
X_L, y_C_L, y_R_L, X_U, y_C_U, y_R_U = split_labeled_unlabeled_with_2_labels(X, y_C, y_R , k=0.2)

原始数据 X 形状: (41188, 62)
原始分类标签 y_C 形状: (41188,)
原始回归标签 y_R 形状: (41188,)
选择 8237 个样本作为有标签数据，剩余样本作为无标签数据
有标签数据 X_L 形状: (8237, 62)
有标签数据 y_C_L 形状: (8237,)
有标签数据 y_R_L 形状: (8237,)
无标签数据 X_U 形状: (32951, 62)
无标签数据 y_C_U 形状: (32951,)
无标签数据 y_R_U 形状: (32951,)


In [20]:
config_clf = {
    'penalty': None,
    'solver': 'lbfgs',
    'max_iter': 100
}

config_reg = {
    'alpha': 1.0,
    'solver': 'auto',
    'max_iter': None
}

In [21]:
clf = LogisticRegression(**config_clf)
reg = Ridge(**config_reg)

In [22]:
two_step = TwoStep(base_classifier=clf, base_regressor=reg,max_iter=100,k=0.1)

2025-01-21 19:36:48,342 - INFO - TwoStep 类已初始化: max_iter=100, k=0.10, base_classifier=LogisticRegression, base_regressor=Ridge


In [23]:
two_step.fit(X_L, y_C_L, X_U)

2025-01-21 19:36:50,219 - INFO - 有标签数据 (L): 8237 样本
2025-01-21 19:36:50,219 - INFO - 无标签数据 (U): 32951 样本
2025-01-21 19:36:50,220 - INFO - 任务类型判定结果: classification
2025-01-21 19:36:50,220 - INFO - 这是一个分类任务。
2025-01-21 19:36:50,221 - INFO - 分类任务中的类别: [0 1]
2025-01-21 19:36:50,222 - INFO - 初始化预测结果数组，长度=32951
2025-01-21 19:36:50,222 - INFO - 开始自训练迭代，共计划进行 100 次迭代
2025-01-21 19:36:50,223 - INFO - ===== 第 0 次迭代开始 =====
2025-01-21 19:36:50,223 - INFO - 当前 Labeled 数据量: 8237；Unlabeled 数据量: 32951
2025-01-21 19:36:50,224 - INFO - 开始训练分类器 (基于 8237 个样本)...
2025-01-21 19:36:50,275 - INFO - 分类器训练完成，耗时 0.05 秒。
2025-01-21 19:36:50,276 - INFO - 分类器识别的类别: [0 1]
2025-01-21 19:36:50,277 - INFO - 开始对 Unlabeled 数据进行置信度打分...
2025-01-21 19:36:50,285 - INFO - 完成置信度打分，耗时 0.01 秒。
2025-01-21 19:36:50,286 - INFO - 选取前 10.00% 的高置信度样本。
2025-01-21 19:36:50,286 - INFO - 本轮选出高置信度样本数量: 3295，占比约为 10.00%
2025-01-21 19:36:50,287 - INFO - 对选中的高置信度样本进行预测标签。
2025-01-21 19:36:50,290 - INFO - 将高置信度样本合并到 Labeled 集。
2025-01-21 19:

In [24]:
two_step.fit(X_L, y_R_L, X_U)

2025-01-21 19:37:58,426 - INFO - 有标签数据 (L): 8237 样本
2025-01-21 19:37:58,427 - INFO - 无标签数据 (U): 32951 样本
2025-01-21 19:37:58,428 - INFO - 任务类型判定结果: regression
2025-01-21 19:37:58,428 - INFO - 这是一个回归任务。
2025-01-21 19:37:58,429 - INFO - 回归任务标签统计: 最小值=-2.8157, 最大值=0.8452, 平均值=0.0127, 标准差=0.9837
2025-01-21 19:37:58,430 - INFO - 初始化预测结果数组，长度=32951
2025-01-21 19:37:58,430 - INFO - 开始自训练迭代，共计划进行 100 次迭代
2025-01-21 19:37:58,430 - INFO - ===== 第 0 次迭代开始 =====
2025-01-21 19:37:58,431 - INFO - 当前 Labeled 数据量: 8237；Unlabeled 数据量: 32951
2025-01-21 19:37:58,431 - INFO - 开始训练回归器 (基于 8237 个样本)...
2025-01-21 19:37:58,443 - INFO - 回归器训练完成，耗时 0.01 秒。
2025-01-21 19:37:58,444 - INFO - 开始对 Unlabeled 数据进行置信度打分...
2025-01-21 19:37:58,451 - INFO - 完成置信度打分，耗时 0.01 秒。
2025-01-21 19:37:58,452 - INFO - 选取前 10.00% 的高置信度样本。
2025-01-21 19:37:58,453 - INFO - 本轮选出高置信度样本数量: 3295，占比约为 10.00%
2025-01-21 19:37:58,453 - INFO - 对选中的高置信度样本进行预测标签。
2025-01-21 19:37:58,454 - INFO - 将高置信度样本合并到 Labeled 集。
2025-01-21 19:37:58,457 -

In [35]:
two_step.get_unlabled_predict('clf').astype('int')

(32951,)

In [34]:
y_C_L.shape

(8237,)

In [36]:
evaluate_model(y_C_U,two_step.get_unlabled_predict('clf').astype('int'))


========== Classification Metrics ==========
Accuracy         : 0.9056
Precision (macro): 0.7984
Recall (macro)   : 0.6466
F1 (macro)       : 0.6880

Confusion Matrix:
[[28677   550]
 [ 2562  1162]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     29227
           1       0.68      0.31      0.43      3724

    accuracy                           0.91     32951
   macro avg       0.80      0.65      0.69     32951
weighted avg       0.89      0.91      0.89     32951



In [37]:
evaluate_model(y_R_U,two_step.get_unlabled_predict('reg'),task='regression')


========== Regression Metrics ==========
MSE               : 0.0050
RMSE              : 0.0710
MAE               : 0.0391
R^2 (Coefficient) : 0.9950
Explained Variance: 0.9950


In [38]:
min_con = None

In [40]:
print("%.2f" % min_con)

TypeError: must be real number, not NoneType